In [1]:
import os
import json
import re
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI

from typing import Any, Dict, List, Literal, Optional
from pydantic import BaseModel, Field, ConfigDict, ValidationError

from plot_json_graphviz import render_json_graph

client = OpenAI()

In [2]:
# variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# models
cheap = 'gpt-4o'
best = 'o3'

# prompts
from prompts import PROMPT_REASONING_STRICT_DEV, PROMPT_JSON_STRICT_DEV, PROMPT_JSON_STRICT_DEV, PROMPT_STRUCT_STRICT_DEV, PROMPT_RESPONSE_EVAL_DEV, PROMPT_REASONING_FLEX_DEV

# io
input_dir = 'input'
json_dir = 'json'
traces_dir = 'traces'
evals_dir = 'evals'

In [ ]:
# run once to upload all papers from /input directory

# all_papers = []

# for paper in os.listdir(input_dir):
#     if not paper.endswith('.pdf'):
#         continue

#     paper_path = os.path.join(input_dir, paper)
    
#     file = client.files.create(
#         file=open(paper_path, "rb"),
#         purpose="user_data"
#     )

#     all_papers.append({
#         "id": paper,
#         "file_id": file.id,
#         "file_name": paper.removesuffix('.pdf'),
#         "file_path": paper_path,
#         "file_size": os.path.getsize(paper_path),
#         "file_created_at": datetime.fromtimestamp(os.path.getctime(paper_path)).isoformat(),
#         "file_modified_at": datetime.fromtimestamp(os.path.getmtime(paper_path)).isoformat(),
#         "file_uploaded_at": datetime.now().isoformat(),
#     })

# # save all_papers to file
# with open(os.path.join('all_papers.json'), 'w') as f:
#     json.dump(all_papers, f, indent=2)

In [3]:
# load all_papers from file
with open(os.path.join('all_papers.json'), 'r') as f:
    all_papers = json.load(f)

In [ ]:
# subset papers
all_papers = all_papers[:3]

In [4]:
# helper functions 

def read_file(file_path, encoding='utf-8'):
    with open(file_path, 'r', encoding=encoding) as f:
        return f.read()
    
# Get json from non-serial (Response) objects
def get_json(obj):
    if hasattr(obj, "__dict__"):
        return {k: get_json(v) for k, v in vars(obj).items()}
    if isinstance(obj, (list, tuple)):
        return [get_json(v) for v in obj]
    if isinstance(obj, dict):
        return {k: get_json(v) for k, v in obj.items()}
    return obj
    
# json extraction
def extract_first_json(text):
    start = text.find('{')
    if start == -1:
        return None
    count = 0
    for i in range(start, len(text)):
        if text[i] == '{':
            count += 1
        elif text[i] == '}':
            count -= 1
            if count == 0:
                return json.loads(text[start:i+1])
    return None

# json validation
def validate_with_paths(model: type[BaseModel], data: Dict[str, Any]) -> Dict[str, Any]:
    try:
        instance = model.model_validate(data)  # Pydantic v2
        return {
            "ok": True,
            "errors": [],
            "instance": instance
        }
    except ValidationError as e:
        def path(loc: List[Any]) -> str:
            parts = []
            for p in loc:
                if isinstance(p, int):
                    parts[-1] = f"{parts[-1]}.{p}"  # attach index to previous name
                else:
                    parts.append(str(p))
            return ".".join(parts)

        errors = [
            {
                "path": path(err.get("loc", [])),
                "msg": err.get("msg", ""),
                "type": err.get("type", "")
            }
            for err in e.errors()
        ]
        return {
            "ok": False,
            "errors": errors
        }

# Save freeform response as txt
def save_reasoning_response(resp: str, file_name: str, model: str): 
    trace_path = os.path.join(traces_dir, f"{file_name}_{model}.txt")
    usage = json.dumps(get_json(resp.usage))

    with open(trace_path, "w", encoding="utf-8") as f:
        f.write(resp.output_text + '\n\n' + usage)

# Save structured response as JSON
def save_json_response(resp: str, file_name: str, model: str):
    if model == 'o3':
        structured_data = json.loads(resp.model_dump()['output'][1]['arguments'])
    else:
        structured_data = json.loads(resp.model_dump()['output'][0]['arguments'])

    structured_json_path = os.path.join(json_dir, f"{file_name}_{model}.json")
    usage = json.dumps(get_json(resp.usage))

    with open(structured_json_path, "w", encoding="utf-8") as f:
        json.dump(structured_data, f, ensure_ascii=False, indent=2)
        f.write('\n\n' + usage)



In [6]:
# pydantic classes

# Schema 1 - Strict
class Node1(BaseModel):
    name: str  # concise natural-language description of node
    aliases: List[str] # 2-3 alternative concise descriptions of node
    type: Literal["concept", "intervention"]
    description: str # detailed technical description of node
    concept_category: Optional[str] = Field(default=None, description="from examples or create a new category (concept nodes only, otherwise null)")
    intervention_maturity: Optional[int] = Field(default=None, ge=1, le=5, description="1-4 (only for intervention nodes)")
    model_config = ConfigDict(extra="forbid")

class Edge1(BaseModel):
    type: str  # relationship label verb
    source_node: str  # source node name
    target_node: str  # target node name
    description: str  # concise description of logical connection
    edge_confidence: int = Field(ge=1, le=5, description="1-5")
    model_config = ConfigDict(extra="forbid")
    
class LogicalChain1(BaseModel):
    title: str  # concise natural-language description of logical chain
    edges: List[Edge1]
    model_config = ConfigDict(extra="forbid")

class PaperSchema1(BaseModel):
    nodes: List[Node1]
    logical_chains: List[LogicalChain1]
    model_config = ConfigDict(extra="forbid")

In [7]:
# Schema 2 - Flex
class Node2(BaseModel):
    type: str
    name: str
    canonical_name: str
    aliases: List[str]
    confidence: float = Field(..., ge=0.0, le=1.0)
    notes: str

class Edge2(BaseModel):
    type: str
    rationale: str
    confidence: float = Field(..., ge=0.0, le=1.0)
    source_node: Node2
    target_node: Node2

class PaperSchema2(BaseModel):
    nodes: List[Node1]
    logical_chains: List[LogicalChain1]
    model_config = ConfigDict(extra="forbid")

In [8]:
# get text response from model
def get_single_response(file_id: str, prompt_text: str, model: str = 'gpt-4.0'):

    input = [{
        "role": "user",
        "content": [
            {"type": "input_file", "file_id": file_id},
            {"type": "input_text", "text": prompt_text}
        ]
    }]
    
    response = client.responses.create(
        model=model,
        input=input,
    )

    return response

In [ ]:
# # get text and structured json in single response -- models won't comply
# def get_single_tool_response(file_id: str, prompt_text: str, schema: object, model: str = 'gpt-4.0'):

#     input = [{
#         "role": "system",
#         "content": "Provide a detailed analysis in text format, then call the causal_chain_structure tool to provide a structured json. Always provide BOTH outputs."
#     }, {
#         "role": "user",
#         "content": [
#             {"type": "input_file", "file_id": file_id},
#             {"type": "input_text", "text": prompt_text}
#         ]
#     }]

#     tools = [{
#         "type": "function",
#         "name": "causal_chain_structure",
#         "description": "Capture the paper's Logical Chain analysis as a structured JSON object.",
#         "parameters": schema.model_json_schema()
#     }]
    
#     response = client.responses.create(
#         model=model,
#         input=input,
#         tools=tools,
#         tool_choice={"type": "allowed_tools", 
#                      "mode": "required",
#                      "tools": [{"type": "function", "name": "causal_chain_structure"}]
#         }
#     )

#     return response

In [9]:
# dual responses from model (reasoning + json in separate requests)
def get_dual_response(file_id: str, prompt_text: str, schema: object, model: str = 'gpt-4.0'):
    """
    Get response from model in two steps:
    1. Freeform analysis of the paper based on the prompt_text.
    2. Structured json using the causal_chain_structure tool based on the freeform analysis.
    """

    # First call - get reasoning
    reasoning_input = [{
        "role": "user",
        "content": [
            {"type": "input_file", "file_id": file_id},
            {"type": "input_text", "text": prompt_text}
        ]
    }]
    
    reasoning_response = client.responses.create(
        model=model,
        input=reasoning_input,
        tools=None  # No tools for reasoning
    )

    # Second call - get structured json
    json_input = [{
        "role": "system",
        "content": "Use the following detailed analysis to help create the structured json:"
    }, {
        "role": "assistant",
        "content": reasoning_response.output_text
    },{
        "role": "user",
        "content": [
            # {"type": "input_file", "file_id": file_id},
            {"type": "input_text", "text": PROMPT_STRUCT_STRICT_DEV}
        ]
    }]

    tools = [{
        "type": "function",
        "name": "causal_chain_structure",
        "description": "Capture the paper's Logical Chain analysis as a structured JSON object.",
        "parameters": schema.model_json_schema()
    }]
    
    json_response = client.responses.create(
        model=model,
        input=json_input,
        tools=tools,
        tool_choice={"type": "allowed_tools", 
                     "mode": "auto",
                     "tools": [{"type": "function", "name": "causal_chain_structure"}]
        }  # force tool use
    )

    return reasoning_response, json_response

In [10]:
# main function to analyze paper
def analyze_paper(file_name: str, file_id: str, prompt_text: str, schema: object, dual: bool = False, label: str = '', model: str = 'gpt-4.0'):
    
    if dual:
        reasoning_response, json_response = get_dual_response(
            file_id=file_id,
            prompt_text=prompt_text,
            schema=schema,
            model=model)
        save_reasoning_response(reasoning_response, file_name + label, model=model)
        save_json_response(json_response, file_name + label, model=model)
        return reasoning_response, json_response
    else:
        response = get_single_response(
            file_id=file_id,
            prompt_text=prompt_text,
            model=model)
        save_reasoning_response(response, file_name + label, model=model)
        return response

Tests

In [ ]:
# # single pass, reasoning only
# file_name = all_papers[2]['file_name']
# file_id = all_papers[2]['file_id']

# resp = analyze_paper(
#     file_name=file_name,
#     file_id=file_id,
#     prompt_text=PROMPT_REASONING_STRICT_DEV,
#     schema=PaperSchema1,
#     model=best,
#     label='_reason'
# )

In [12]:
# single pass, reasoning + json
file_name = all_papers[2]['file_name']
file_id = all_papers[2]['file_id']

resp = analyze_paper(
    file_name=file_name,
    file_id=file_id,
    prompt_text=PROMPT_REASONING_STRICT_DEV + PROMPT_JSON_STRICT_DEV,
    schema=PaperSchema1,
    model=best,
    label='_strict_final_1p'
)

In [ ]:
# # single pass with struct -- models tend to call tool only

# file_name = all_papers[2]['file_name']
# file_id = all_papers[2]['file_id']

# response = get_single_tool_response(
#         file_id=file_id,
#         prompt_text=PROMPT_REASONING_STRICT_DEV+PROMPT_STRUCT_STRICT_DEV,
#         schema=PaperSchema1,
#         model=best
#         )

# resp = get_json(response)
# print(json.loads(resp['output'][1]['arguments']))
# print(response.output_text)

In [ ]:
# # dual pass
# file_name = all_papers[2]['file_name']
# file_id = all_papers[2]['file_id']

# resp = analyze_paper(
#     file_name=file_name,
#     file_id=file_id,
#     prompt_text=PROMPT_REASONING_STRICT_DEV,
#     schema=PaperSchema1,
#     dual = True,
#     model=best,
#     label='_2pass'
# )

In [ ]:
# extract all papers, single-pass

# for paper in all_papers:
#     analyze_paper(
#         file_name=paper['file_name'],
#         file_id=paper['file_id'],
#         prompt_text=PROMPT_REASONING_STRICT_DEV + PROMPT_JSON_STRICT_DEV,
#         schema=PaperSchema1,
#         model=best,
#         label='_strict_1p'
#     )


# for paper in all_papers:
#     analyze_paper(
#         file_name=paper['file_name'],
#         file_id=paper['file_id'],
#         prompt_text=PROMPT_REASONING_FLEX_DEV,
#         schema=PaperSchema2,
#         model=best,
#         label='_flex_1p'
#     )

In [ ]:
# extract all papers, dual-pass

# for paper in all_papers:
#     analyze_paper(
#         file_name=paper['file_name'],
#         file_id=paper['file_id'],
#         prompt_text=PROMPT_REASONING_STRICT_DEV,
#         schema=PaperSchema1,
#         dual=True,
#         model=best,
#         label='_strict_2p'
#     )

-----

## evals

In [ ]:
# load dual-pass 
dual_json_data = {}

for filename in os.listdir(json_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(json_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            json_part = extract_first_json(content)
            dual_json_data[filename] = json_part

In [ ]:
for key in dual_json_data.keys():
    print(validate_with_paths(PaperSchema1, dual_json_data[key]))

In [ ]:
# load single-pass
single_json_data = {}
keyword = 'flex'

for filename in sorted(os.listdir(traces_dir)):
    if keyword in filename:
        file_path = os.path.join(traces_dir, filename)
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
            json_part = extract_first_json(content)
            single_json_data[filename] = json_part


In [ ]:
for key in single_json_data.keys():
    print(validate_with_paths(PaperSchema2, single_json_data[key]))

In [ ]:
# check
single_json_data.keys()

# due to a new type, validation schema format incorrect?

In [ ]:
# reasoning evals

In [ ]:
# llm-as-judge, feed paper and concat responses
def eval_analyses(file_id: str, eval_text: str, prompt_text: str, model: str = 'gpt-4.0'):

    input = [{
        "role": "user",
        "content": [
            {"type": "input_file", "file_id": file_id},
            {"type": "input_text", "text": prompt_text},
            {"type": "input_text", "text": eval_text},
        ]
    }]

    response = client.responses.create(
        model=model,
        input=input,
    )

    return response
    

In [ ]:
# eval 1p vs 2p
for paper in all_papers:

    file_name = paper['file_name']
    file_id = paper['file_id']

    run1 = '_strict_1p_o3'
    run2 = '_strict_2p_o3'

    label = '_1p2p'

    eval_1 = f"Analysis 1: {file_name}{run1}\n\n" + read_file(os.path.join(traces_dir, f"{file_name}{run1}.txt")) + "\n"
    eval_2 = f"Analysis 2: {file_name}{run2}\n\n" + read_file(os.path.join(traces_dir, f"{file_name}{run2}.txt")) + "\n" + read_file(os.path.join(json_dir, f"{file_name}{run2}.json")) + "\n"
    eval_text = eval_1 + eval_2

    resp = eval_analyses(
        file_id=file_id,
        prompt_text=PROMPT_RESPONSE_EVAL_DEV,
        eval_text=eval_text,
        model=best
    )

    with open(os.path.join(evals_dir, f"{file_name}_{label}_eval.md"), "w", encoding="utf-8") as f:
        f.write(resp.output_text)

In [ ]:
# eval strict 1p vs flex 1p

for paper in all_papers:

    file_name = paper['file_name']
    file_id = paper['file_id']

    run1 = '_strict_1p_o3'
    run2 = '_flex_1p_o3'

    label = '_fs'

    eval_1 = f"Analysis 1: {file_name}{run1}\n\n" + read_file(os.path.join(traces_dir, f"{file_name}{run1}.txt")) + "\n"
    eval_2 = f"Analysis 2: {file_name}{run2}\n\n" + read_file(os.path.join(traces_dir, f"{file_name}{run2}.txt")) + "\n"
    eval_text = eval_1 + eval_2

    resp = eval_analyses(
        file_id=file_id,
        prompt_text=PROMPT_RESPONSE_EVAL_DEV,
        eval_text=eval_text,
        model=best
    )

    with open(os.path.join(evals_dir, f"{file_name}_{label}_eval.md"), "w", encoding="utf-8") as f:
        f.write(resp.output_text)

-----

In [ ]:
# iterative analysis
def analyze_paper_iteratively(file_name: str, file_id: str, prompt_text: str, iterations: int = 3, schema: object, model: str = 'gpt-4.0'):
    """
    Iteratively analyze a paper multiple times, asking the model to find more connections each time.
    
    Args:
        file_name (str): Name of the file to analyze.
        file_id (str): ID of the file in OpenAI.
        prompt_text (str): The base prompt to use for analysis.
        iterations (int): Number of iterations to perform (default 3).
        model (str): The model to use for analysis.
    
    Returns:
        List of tuples containing (freeform_response, structured_response) for each iteration.
    """
    
    results = []
    current_prompt = prompt_text
    
    for i in range(iterations):
        print(f"\nIteration {i+1}/{iterations}")
        
        # For iterations after the first, add the improvement request
        if i > 0:
            current_prompt = (
                current_prompt + 
                "\n\nIMPORTANT: You missed many causal connections and relationships in your previous analysis. " +
                "Please analyze again more thoroughly, looking specifically for:\n" +
                "1. Additional connections between existing concepts\n" +
                "2. Implicit relationships that weren't directly stated\n" +
                "3. Higher-order effects and consequences\n" +
                "4. Cross-cutting themes and patterns\n" +
                "5. Alternative interpretations of the findings"
            )
        
        # Run the analysis
        freeform_response, structured_response = get_dual_response(
            file_id=file_id,
            prompt_text=current_prompt,
            schema=schema,
            model=model
        )
        
        # Save responses with iteration number in filename
        save_trace_response(freeform_response, f"{file_name}_iter{i+1}", model=model)
        save_json_response(structured_response, f"{file_name}_iter{i+1}", model=model)
        
        results.append((freeform_response, structured_response))
        
    return results

In [ ]:
# test iterative analysis
file_name = all_papers[0]['file_name']
file_id = all_papers[0]['file_id']

iterative_results = analyze_paper_iteratively(
    file_name=file_name,
    file_id=file_id,
    prompt_text=PROMPT_FREEFORM,
    iterations=3,
    schema=PaperSchema1,
    model=cheap
)

# Print the freeform responses from each iteration
# for i, (freeform_resp, _) in enumerate(iterative_results, 1):
#     print(f"\n=== Iteration {i} Analysis ===")
#     print(freeform_resp.output_text)